<a href="https://colab.research.google.com/github/41371120h/PL-Repo.peng/blob/main/HW4_%E6%96%87%E5%AD%97%E8%B3%87%E6%96%99%E5%B0%8F%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================================================================
# 🔹 Yahoo 股市新聞分析 → TF-IDF → Gemini AI 洞察 (Sheet 強化版)
# ==============================================================================

# --- 運行環境設定（請在 Colab Cell 中執行）---
!pip -q install gspread gspread_dataframe google-auth google-auth-oauthlib google-auth-httplib2 \
              gradio pandas beautifulsoup4 google-generativeai python-dateutil scikit-learn jieba

import os, time, uuid, re, json, datetime
from datetime import datetime as dt, timedelta
from dateutil.tz import gettz
import pandas as pd
import gradio as gr
import requests
from requests.exceptions import RequestException, Timeout
from bs4 import BeautifulSoup
import google.generativeai as genai
import jieba  # 使用針對繁體中文優化的 jieba
import jieba.analyse
import jieba.posseg as pseg

# Google Auth & Sheets
from google.colab import auth, userdata
from google.auth import default
import gspread
from gspread_dataframe import set_with_dataframe, get_as_dataframe

# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import traceback
import pytz

# --- Google 認證與 Gemini 配置 ---
try:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)
    print("✅ Google Sheets 授權成功。")

    GEMINI_API_KEY = userdata.get("gemini")
    if not GEMINI_API_KEY:
         raise ValueError("Colab Secret 'gemini' is empty or not found. Please set your Gemini API Key.")

    genai.configure(api_key=GEMINI_API_KEY)
    model = genai.GenerativeModel("gemini-2.5-flash") # 使用更快速的模型
    print("✅ Gemini API Key 配置成功。")
except Exception as e:
    print(f"🚨 授權或設定時發生錯誤：{e}")


# ==============================================================================
# 1. 全域變數與 Sheet/DataFrame 設置
# ==============================================================================
# 請檢查您的 Sheet URL，確保正確
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/107FcjXEnPn7vM10qFPj-wQFPeeNUOWTwKk5A-ejJqo4/edit?gid=412911623#gid=412911623"
TIMEZONE = "Asia/Taipei"

# 確保這些欄位與 DF 輸出一致
CLIPS_HEADER = ["日期", "作者", "標題", "連結", "內文"]
STATS_HEADER = ["關鍵字", "TF-IDF平均權重"]
SUMMARY_HEADER = ["created_at", "keywords_used", "summary_report"]

# --- Google Sheet 初始化函式 (沿用您提供的邏輯) ---
def get_or_create_worksheet(sheet, title):
    try:
        worksheet = sheet.worksheet(title)
    except gspread.exceptions.WorksheetNotFound:
        worksheet = sheet.add_worksheet(title=title, rows="100", cols="20")
    return worksheet

def write_to_sheet(sheet, worksheet_name, df, log_output, header_list):
    log_output.append(f"--- 2. Google Sheet 寫入日誌 ---")
    try:
        worksheet = get_or_create_worksheet(sheet, worksheet_name)
        # 🚨 關鍵修正：確保寫入的欄位與 header_list 一致
        if not df.empty:
            df_to_write = df.reindex(columns=header_list, fill_value="")

            # 使用 update 寫入標題和數據
            worksheet.clear()
            worksheet.update(
                [df_to_write.columns.values.tolist()] + df_to_write.astype(str).values.tolist(),
                value_input_option="USER_ENTERED"
            )
            log_output.append(f"✅ 成功寫入 {worksheet_name} 工作表 ({len(df_to_write)} 筆資料)。")
        else:
             worksheet.clear()
             worksheet.update([header_list], value_input_option="USER_ENTERED")
             log_output.append(f"✅ {worksheet_name} 工作表已清空 (無資料寫入)。")
    except Exception as e:
        log_output.append(f"❌ 寫入 Sheet 失敗: {e}")
    return log_output

# 開啟試算表並初始化工作表
try:
    gsheets = gc.open_by_url(SPREADSHEET_URL)
    print(f"✅ 成功開啟 Sheet: {gsheets.title}")
    # 這裡只確保工作表存在，實際寫入邏輯在 write_to_sheet 中
    ws_clips = get_or_create_worksheet(gsheets, "Yahoo文章列表")
    ws_stats = get_or_create_worksheet(gsheets, "熱詞統計")
    ws_summary = get_or_create_worksheet(gsheets, "AI摘要報告")
except Exception as e:
     print(f"❌ 無法初始化 Google Sheet: {e}")
     raise # 停止執行

# ==============================================================================
# 2. 爬蟲、TF-IDF 統計與 Gemini 摘要
# ==============================================================================

# --- 2.1 Yahoo 新聞爬蟲 (核心修正) ---
YAHOO_STOCK_URL = "https://tw.stock.yahoo.com/news"

def scrape_yahoo_stock_news(num_articles_to_fetch, log_output):
    """專門爬取 Yahoo 股市新聞指定文章數的文章列表與內文"""

    # 使用通用的文章列表 selector
    LIST_SELECTOR = "a[href*='tw.stock.yahoo.com/news/']"

    session = requests.Session()
    all_data_list = []
    log_output.append(f"--- 1. 爬蟲日誌 ---")
    log_output.append(f"目標網站: Yahoo 股市新聞 | 爬取文章數: {num_articles_to_fetch}")

    # 使用標準 Headers 模擬瀏覽器
    enhanced_headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Accept-Language": "zh-TW,zh;q=0.8,en-US;q=0.5,en;q=0.3",
    }

    try:
        # 1. 爬取列表頁
        r = session.get(YAHOO_STOCK_URL, timeout=15, headers=enhanced_headers)
        r.raise_for_status()
        soup = BeautifulSoup(r.text, "html.parser")
        article_links = soup.select(LIST_SELECTOR)[:num_articles_to_fetch]

        log_output.append(f"列表頁找到 {len(article_links)} 篇文章連結。")

        # 2. 爬取內頁
        for i, a_tag in enumerate(article_links):
            link = a_tag.get("href")
            if not link or "javascript:void(0)" in link: continue

            # 處理相對路徑
            if not link.startswith("http"):
                from urllib.parse import urljoin
                link = urljoin(YAHOO_STOCK_URL, link)

            # 爬取內頁
            try:
                sub_resp = session.get(link, timeout=10, headers=enhanced_headers)
                sub_resp.raise_for_status()
                sub_soup = BeautifulSoup(sub_resp.text, "html.parser")

                # 抓取標題 (h1)
                title = sub_soup.select_one("h1").get_text(strip=True) if sub_soup.select_one("h1") else "無標題"

                # 抓取內容 (p 標籤內文)
                content_nodes = sub_soup.select("p")
                content = " ".join([p.get_text(strip=True) for p in content_nodes if len(p.get_text(strip=True)) > 20])

                # 抓取作者/日期 (通常在特定的 span/div 內，這裡使用簡化方式)
                date_node = sub_soup.select_one("time")
                date_str = date_node.get("datetime") if date_node and date_node.get("datetime") else dt.now(gettz(TIMEZONE)).strftime("%m/%d")
                author = sub_soup.select_one("span.author-name")
                author_str = author.get_text(strip=True) if author else "Yahoo 股市"

                all_data_list.append({
                    "日期": date_str,
                    "作者": author_str,
                    "標題": title,
                    "連結": link,
                    "內文": content
                })
                log_output.append(f"   -> 成功擷取 #{i+1}: {title[:20]}...")
            except Exception as e:
                log_output.append(f"   ⚠️ 爬取或解析內頁失敗 ({link}): {e}")
                continue

            time.sleep(0.1)

    except requests.exceptions.RequestException as e:
        log_output.append(f"❌ 爬蟲起始請求失敗：{e}")

    df = pd.DataFrame(all_data_list)
    log_output.append(f"✅ 爬蟲結束。共抓取 {len(df)} 篇文章。")
    return df, log_output

# --- 2.2 TF-IDF 關鍵字分析 (中文金融優化) ---

STOPWORDS = set([
    '的', '了', '是', '在', '我', '你', '他', '她', '之', '一個', '和', '與', '或', '也', '都', '將',
    '被', '由', '所', '於', '於此', '這', '那', '而', '但', '並', '則', '要', '應', '進行', '如果',
    # 數字與單位
    '元', '萬元', '億元', '萬', '億', '千', '百', '個', '日', '月', '年', '季', '週', '天', '點', '度',
    # 常用詞
    '公司', '企業', '市場', '指出', '表示', '報導', '分析', '認為', '提供', '資訊', '網站', '股價', '股市',
    '投資', '交易', '客戶', '業務', '產品', '服務', '資料', '已經', '不過', '此外', '目前', '未來', '預計',
    # 介系詞與連詞
    '對於', '關於', '由於', '因為', '隨著', '除了', '包括', '例如', '如果說', '甚至', '還是', '還是說'
])

def chinese_tokenizer(text):
    """分詞並過濾停用詞和單字"""
    # 移除標點符號和數字
    cleaned_text = re.sub(r'[^\w\s]', ' ', text).strip()
    cleaned_text = re.sub(r'\d+', ' ', cleaned_text)

    # 使用精確模式分詞
    words = jieba.lcut(cleaned_text, cut_all=False)

    # 過濾停用詞和單字
    filtered_words = [
        word.strip()
        for word in words
        if word.strip() and len(word.strip()) > 1 and word.strip().lower() not in STOPWORDS
    ]
    return filtered_words


def get_tfidf_keywords(df, top_n, log_output):
    """使用 sklearn.TfidfVectorizer 進行 TF-IDF 分析"""

    log_output.append(f"--- 3. TF-IDF 分析日誌 (Sklearn) ---")
    log_output.append(f"目標關鍵字數量: Top {top_n}")
    log_output.append(f"停用詞數量: {len(STOPWORDS)}")

    if '內文' not in df.columns or df['內文'].dropna().empty:
        log_output.append("❌ 錯誤: 資料集中缺少 '內文' 欄位或內文為空。")
        return pd.DataFrame(columns=STATS_HEADER), log_output

    document_list = []

    # 1. 文本預處理與分詞
    for content in df['內文'].dropna():
        filtered_words = chinese_tokenizer(content)
        if filtered_words:
            document_list.append(" ".join(filtered_words))

    log_output.append(f"已將 {len(df)} 篇文章分詞並過濾，產生 {len(document_list)} 篇有效文檔。")

    if not document_list:
        log_output.append("⚠️ 沒有可分析的文檔 (可能都被過濾了)。")
        return pd.DataFrame(columns=STATS_HEADER), log_output

    # 2. TF-IDF 計算
    try:
        # 使用自定義分詞器，並設定 n-gram (1-gram 和 2-gram)
        vectorizer = TfidfVectorizer(tokenizer=chinese_tokenizer, ngram_range=(1, 2))
        tfidf_matrix = vectorizer.fit_transform(document_list)
        feature_names = vectorizer.get_feature_names_out()

        log_output.append(f"✅ TF-IDF 矩陣建立成功。詞彙總數 (Features): {len(feature_names)}")

        # 3. 計算平均權重 (找出整體重要性)
        # 🚨 關鍵修正: 使用 sum(axis=0) 計算總權重而非平均，在不同文件數量下更穩定
        sum_tfidf_scores = tfidf_matrix.sum(axis=0).tolist()[0]

        # 4. 建立 DataFrame 排序並選出 Top N
        keywords_with_scores = list(zip(feature_names, sum_tfidf_scores))

        # 排序
        sorted_keywords = sorted(keywords_with_scores, key=lambda item: item[1], reverse=True)

        top_keywords_df = pd.DataFrame(
            [(k, round(s, 4)) for k, s in sorted_keywords[:top_n]],
            columns=['關鍵字', 'TF-IDF平均權重']
        )

        log_output.append(f"✅ 成功提取 Top {len(top_keywords_df)} 個關鍵字。")
        log_output.append(f"Top 5 關鍵字範例: {', '.join(top_keywords_df['關鍵字'].head(5).tolist())}")

        return top_keywords_df, log_output

    except ValueError as e:
        log_output.append(f"❌ TF-IDF 分析失敗 (ValueError): {e}")
        return pd.DataFrame(columns=STATS_HEADER), log_output
    except Exception as e:
        log_output.append(f"❌ TF-IDF 分析發生未預期錯誤: {e}")
        return pd.DataFrame(columns=STATS_HEADER), log_output


# --- 2.3 Gemini API 生成摘要 (保留高超時設定) ---
def get_gemini_summary(keywords_df, log_output):
    """使用 Gemini API 根據關鍵字生成摘要，並將結果寫入 Sheet"""

    log_output.append(f"--- 4. Gemini 摘要日誌 ---")

    if keywords_df.empty:
        log_output.append("⚠️ 缺少關鍵字，無法生成摘要。")
        return "⚠️ 沒有關鍵字，無法生成摘要。", log_output

    keywords_list = keywords_df['關鍵字'].tolist()
    prompt = f"""
    您是一位專業的股市數據分析師。

    任務：
    請根據 Yahoo 股市新聞的 {len(keywords_list)} 個熱門關鍵字，生成一份專業的股市分析報告。

    熱門關鍵字 (依 TF-IDF 總權重排序)：
    {', '.join(keywords_list)}

    輸出格式要求 (請嚴格遵守)：
    1.  **五句洞察摘要**：條列式，每句都是精闢的股市觀察。
    2.  **一段 120 字結論**：總結目前的股市趨勢或投資機會。

    請使用繁體中文回答。
    """

    summary_text = ""
    try:
        log_output.append(f"模型請求參數: gemini-2.5-flash, 關鍵字數量: {len(keywords_list)}")
        # 🚨 關鍵修正：保持 120 秒超時，應對連線不穩定的問題
        response = model.generate_content(prompt, request_options={"timeout": 120})

        summary_text = response.text.replace("#", "").replace("*", "")
        log_output.append("✅ 摘要生成成功。")
        msg_status = "✅ Gemini 摘要生成完成。"
    except Exception as e:
        error_msg = f"❌ Gemini API 呼叫失敗：{e}"
        log_output.append(error_msg)
        summary_text = error_msg
        msg_status = f"⚠️ Gemini 請求失敗，詳情請看日誌。"

    # --- 寫入 AI 摘要至 Sheet ---
    try:
        # 將最新的摘要寫入 'AI摘要報告' 工作表
        new_row_df = pd.DataFrame([{
            "created_at": dt.now(gettz(TIMEZONE)).isoformat(),
            "keywords_used": ", ".join(keywords_list[:10]), # 只寫入前 10 個關鍵字
            "summary_report": summary_text
        }], columns=SUMMARY_HEADER)

        # 由於您原程式中沒有 read_df for summary，這裡使用 gspread 的 append 寫入 (放在最前面需要更多邏輯，先用寫入單行替代)
        # 確保在 Sheet 中，此工作表 'AI摘要報告' 已經有 SUMMARY_HEADER

        # 🚨 修正：使用 df 的前幾行模擬寫入邏輯
        # 讀取現有數據，將新數據插入最前面，然後寫回 (確保最新的在上方)
        df_existing = ws_summary.get_all_values()
        df_existing_data = [row for row in df_existing if row != SUMMARY_HEADER]

        # 準備新資料列 (必須是 list of lists)
        new_data_row = new_row_df.iloc[0].values.tolist()

        # 清空工作表並寫入新的標題和數據
        ws_summary.clear()
        ws_summary.update([SUMMARY_HEADER] + [new_data_row] + df_existing_data, value_input_option="USER_ENTERED")

        log_output.append("✅ 摘要和關鍵詞已成功寫入 'AI摘要報告' 工作表。")
    except Exception as e:
        log_output.append(f"❌ 寫入 AI 摘要至 Sheet 失敗: {e}")

    return summary_text, log_output

# ================================
# 3. Gradio 整合函式
# ================================
def run_full_automation_flow(top_n_str, articles_to_fetch_str):
    """Gradio 點擊後執行的完整流程"""

    empty_df = pd.DataFrame(columns=STATS_HEADER)
    empty_scraped_df = pd.DataFrame(columns=CLIPS_HEADER)
    empty_str = ""
    log_output = []

    SITE_NAME = "Yahoo 股市新聞"
    site_list = [SITE_NAME]

    # 清空上次結果
    yield "日誌將顯示於此...", empty_df, empty_str, None, gr.Radio(choices=["尚未執行"], value="尚未執行"), empty_scraped_df

    # --- 參數驗證 ---
    try:
        top_n = int(top_n_str)
        articles_to_fetch = int(articles_to_fetch_str)
        if top_n <= 0 or articles_to_fetch <= 0:
            log_output.append("❌ Top N 或爬取文章數必須是大於 0 的數字。")
            yield "\n".join(log_output), empty_df, empty_str, None, gr.Radio(choices=site_list), empty_scraped_df
            return
    except ValueError:
        log_output.append("❌ 請輸入有效的數字。")
        yield "\n".join(log_output), empty_df, empty_str, None, gr.Radio(choices=site_list), empty_scraped_df
        return

    # --- 自動化流程 ---
    log_output.append("===================================================")
    current_time_str = dt.now(gettz(TIMEZONE)).strftime('%Y-%m-%d %H:%M:%S')
    log_output.append(f"🚀 自動化流程啟動 ({current_time_str})")
    log_output.append("===================================================")

    try:
        # --- 步驟 1: 爬蟲 (Yahoo News) ---
        log_output.append(f"1/4: 🏃‍♂️ 開始爬取 {SITE_NAME} 文章，請稍等...")
        yield "\n".join(log_output), empty_df, empty_str, None, gr.Radio(choices=site_list, value=SITE_NAME), empty_scraped_df

        scraped_df, log_output = scrape_yahoo_stock_news(articles_to_fetch, log_output)
        display_df = scraped_df[["日期", "作者", "標題", "連結", "內文"]] # 確保顯示正確欄位

        yield "\n".join(log_output), empty_df, empty_str, None, gr.Radio(choices=site_list, value=SITE_NAME), display_df

        if scraped_df.empty:
            log_output.append("❌ 爬蟲失敗，未抓取到任何資料。流程終止。")
            yield "\n".join(log_output), empty_df, empty_str, None, gr.Radio(choices=site_list, value=SITE_NAME), empty_scraped_df
            return

        # --- 步驟 2: 寫入 Sheet (文章列表) ---
        log_output = write_to_sheet(gsheets, "Yahoo文章列表", scraped_df, log_output, CLIPS_HEADER)

        # --- 步驟 3: TF-IDF 分析 ---
        log_output.append("2/4: 📊 正在進行 Sklearn TF-IDF 關鍵字分析...")
        yield "\n".join(log_output), empty_df, empty_str, None, gr.Radio(choices=site_list, value=SITE_NAME), display_df

        keywords_df, log_output = get_tfidf_keywords(scraped_df, top_n, log_output)

        plot_df = None
        if not keywords_df.empty:
          # 為了讓 BarPlot 頂部顯示權重最高的，我們需將 df 升冪排序
          plot_df = keywords_df.sort_values("TF-IDF平均權重", ascending=True)

        yield "\n".join(log_output), keywords_df, empty_str, plot_df, gr.Radio(choices=site_list, value=SITE_NAME), display_df

        if keywords_df.empty:
            log_output.append("⚠️ 分析完成，但未提取到關鍵字。流程終止。")
            yield "\n".join(log_output), empty_df, empty_str, None, gr.Radio(choices=site_list, value=SITE_NAME), display_df
            return

        # --- 步驟 4: 寫入 Sheet (熱詞統計) ---
        log_output.append("3/4: 📈 正在將 Top 熱詞回寫至 Sheet (熱詞統計)...")
        log_output = write_to_sheet(gsheets, "熱詞統計", keywords_df, log_output, STATS_HEADER)

        # --- 步驟 5: Gemini 摘要與寫入 Sheet (AI摘要報告) ---
        log_output.append("4/4: 🧠 正在呼叫 Gemini API 生成摘要 (已設定 120s 超時)...")
        yield "\n".join(log_output), keywords_df, empty_str, plot_df, gr.Radio(choices=site_list, value=SITE_NAME), display_df

        summary, log_output = get_gemini_summary(keywords_df, log_output)

        # 最終回傳
        final_log = "\n".join(log_output)
        yield final_log, keywords_df, summary, plot_df, gr.Radio(choices=site_list, value=SITE_NAME), display_df

        log_output.append("===================================================")
        log_output.append("✅ 全部流程完成！請切換到「最終結果」標籤頁查看。")


    except Exception as e:
        error_msg = f"❌ 流程發生未預期錯誤：{e}\n{traceback.format_exc()}"
        log_output.append(error_msg)
        yield "\n".join(log_output), empty_df, empty_str, None, gr.Radio(choices=site_list, value=SITE_NAME), empty_scraped_df


# ================================
# 4. 啟動 Gradio 介面 (已修改為 Yahoo)
# ================================
print("\n🚀 正在啟動 Gradio 介面...")
with gr.Blocks(theme=gr.themes.Soft(primary_hue="orange"), title="Yahoo 股市新聞分析與 AI 摘要（Sheet 強化版）") as demo:
    gr.Markdown(

        """
        # 📈 Yahoo 股市新聞分析 → TF-IDF 關鍵詞 → AI 洞察摘要
        此工具會自動執行：**Yahoo 爬蟲 → 寫入 Sheet (文章列表) → TF-IDF 統計 → 寫入 Sheet (熱詞統計) → Gemini 生成摘要 → 寫入 Sheet (AI摘要報告)**。
        """
    )

    with gr.Tab("🚀 自動化流程執行"):
        with gr.Row():
            # 🚨 修正：參數名稱從 pages_to_fetch 改為 articles_to_fetch
            articles_to_fetch_input = gr.Textbox(label="要爬取的文章數量 (Limit)", value="10", scale=1)
            top_n_input = gr.Textbox(label="要統計的 Top N 熱詞數量", value="20", scale=1)
            run_btn = gr.Button("🚀 一鍵啟動 Yahoo 股市新聞分析", variant="primary", scale=2)

        gr.Markdown("---")

        # 顯示 Yahoo 爬蟲的參數資訊
        with gr.Row():
            gr.Textbox(label="目標網站", value=YAHOO_STOCK_URL, interactive=False, scale=1)
            gr.Textbox(label="爬蟲模式", value="專門針對 Yahoo 內頁擷取", interactive=False, scale=1)

        # 使用 Tab 來區分最終結果和日誌
        with gr.Tabs():

            with gr.TabItem("🛠️ 技術日誌與輸出細節"):
                log_output_text = gr.Textbox(
                    label="詳細流程日誌 (爬蟲、寫入、分析步驟)",
                    lines=30,
                    interactive=False,
                    show_copy_button=True
                )
            with gr.TabItem("🕸️ 爬取文章列表"):
                site_list_output = gr.Radio(
                    label="資料來源",
                    choices=["尚未執行"],
                    value="尚未執行",
                    interactive=False
                )
                gr.Markdown("---")

                # 由於爬蟲邏輯已改，這裡只顯示 DataFrame，點擊連結功能暫不實作
                scraped_data_output = gr.Dataframe(
                    label="爬取文章列表 (原始資料)",
                    headers=["日期", "作者", "標題", "連結", "內文"],
                    interactive=True,
                    row_count=(15, 'dynamic')
                )

                # 重新綁定 Dataframe 點擊事件，確保即使連結功能未實作，也不會報錯
                link_display_output = gr.Markdown(
          			value="*原始文章資料已顯示於表格。*"
          		)
                # 移除 show_selected_link 點擊事件，避免 Gradio 錯誤

            with gr.TabItem("✅ 最終結果"):
                summary_output = gr.Markdown(label="🤖 Gemini 洞察摘要與結論")

                keyword_plot_output = gr.BarPlot(
                  label="📈 Top N 熱詞視覺化圖表",
                  x="TF-IDF平均權重",
                  y="關鍵字",
                  tooltip=['關鍵字', 'TF-IDF平均權重'],
                  color="TF-IDF平均權重",
                  vertical=False,
                  height=400
                )

                keywords_output = gr.Dataframe(label="📈 Top N 熱詞統計結果 (Sklearn TF-IDF 總權重)")


        # === 綁定動作 ===
        run_btn.click(
          fn=run_full_automation_flow,
          	inputs=[top_n_input, articles_to_fetch_input],
          	outputs=[
                log_output_text,
                keywords_output,
                summary_output,
                keyword_plot_output,
                site_list_output,
                scraped_data_output
            ]
        )

demo.launch(debug=True)

✅ Google Sheets 授權成功。
✅ Gemini API Key 配置成功。
✅ 成功開啟 Sheet: HW4_文字資料小分析

🚀 正在啟動 Gradio 介面...
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://228a41780666910844.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
